In [1]:
# make sure working directory is root of repository
%cd ..
%pwd

/Users/benwang/Desktop/wustl_grad/2022Spring/DCDS500/dcds-500-project-2


'/Users/benwang/Desktop/wustl_grad/2022Spring/DCDS500/dcds-500-project-2'

In [2]:
from shopping_environment import ShoppingEnvironment
import pandas as pd

## Setup Environment

In [3]:
# parameters
num_shoes        = 1000

# number of each consumer type
num_average     = 9000
num_abnormal    = 1000
num_special     = 250
num_wealthy     = 1000
num_influencers = 500
num_resellers   = 2500
num_total       = num_average + num_abnormal + num_special + num_wealthy + num_influencers + num_resellers

# initialize environment
env = ShoppingEnvironment(
    mean_desire = 0.5,
    std_desire = 0.2,
    desire_threshold = 0.5,
    average_mean_money = 250,
    average_std_money = 15,
    rich_mean_money = 1000,
    rich_std_money = 250,
    average_prob_loyal = 0.5,
    rich_prob_loyal = 0.8,
    reseller_prob_loyal = 0.9,
    num_shoes = num_shoes,
    price = 250,
    num_average = num_average,
    num_abnormal = num_abnormal,
    num_special = num_special,
    num_wealthy = num_wealthy,
    num_influencers = num_influencers,
    num_resellers = num_resellers
)

In [4]:
def get_utilities(df):
    # buyer and seller utility for each simulation
    got_shoes = df[df['shoes_acquired'] >= 1]
    print("Consumer utility: ", got_shoes['desire'].mean())     # buyer utility is average desire of those who purchased shoes
    print("Seller utility: ", got_shoes['influence'].mean())    # seller utility is average influence of those who purchased shoes 


def group_fairness(df):
    # Difference in number of shoes purchased by group and expected number of shoes group should get
    ret_dict = {}
    got_shoes = df[df['shoes_acquired'] >= 1]
    ret_dict['Consumer utility'] = got_shoes['desire'].mean()
    ret_dict['Seller utility'] = got_shoes['influence'].mean()
    amount_con_types = {
        'average': num_average,
        'abnormal': num_abnormal,
        'special': num_special,
        'wealthy': num_wealthy,
        'influencer': num_influencers,
        'reseller': num_resellers
    }
    for con, amount in amount_con_types.items():
        ret_dict[f'{con}_num_got_shoes'] = len(got_shoes[got_shoes['identity'] == con])
        ret_dict[f'{con}_percent_got_shoes'] = len(got_shoes[got_shoes['identity'] == con]) / amount
        ret_dict[f'{con}_shoes_bought'] = df.loc[df['identity'] == con, 'shoes_acquired'].sum()
        ret_dict[f'{con}_shoes_per_person'] = df.loc[df['identity'] == con, 'shoes_acquired'].sum() / amount

    return ret_dict

        # print(f'{con} difference in actual: ', len(got_shoes[got_shoes['identity'] == con]))
        # print(f'{con} percent: ', len(got_shoes[got_shoes['identity'] == con]) / amount)
        # print(f'{con} shoes bought: ', df.loc[df['identity'] == con, 'shoes_acquired'].sum())
        # print(f'{con} shoes per person: ', df.loc[df['identity'] == con, 'shoes_acquired'].sum() / amount)
        

## No Gaming

### Lottery
Every consumer has an equal chance of being selected to purchase shoes

In [5]:
env.restock(1000)
env.run_lottery_no_gaming()  # run lottery and get dataframe of all results
df = env.get_consumer_df()
#get_utilities(df)
lottery_no_gaming = pd.Series(group_fairness(df), name='lottery_no_gaming')

### First Come First Served
With no gaming, those with highest desire "come first", and so those with the most desire get the sheos

In [6]:
env.restock(num_shoes = 1000)        # reset environment to 100 shoes with no one purchased yet
env.run_first_come_no_gaming()
df = env.get_consumer_df()
#get_utilities(df)
#group_fairness(df)
first_no_gaming = pd.Series(group_fairness(df), name='first_no_gaming')

### Invitation Only
All influencers are invited, and the rest of the spots are randomly selected from those with loyalty status

In [7]:
env.restock(num_shoes = 1000)
env.run_invitation_no_gaming()
df = env.get_consumer_df()
#get_utilities(df)
invite_no_gaming = pd.Series(group_fairness(df), name='invite_no_gaming')

In [8]:
no_gaming = pd.DataFrame([lottery_no_gaming, first_no_gaming, invite_no_gaming])

## With Gaming

### Lottery
Every consumer has an equal chance of being selected to purchase shoes

In [9]:
env.restock(1000)
env.run_lottery_with_gaming()  # run lottery and get dataframe of all results
df = env.get_fake_df()
#get_utilities(df)
#group_fairness(df)
lottery_with_gaming = pd.Series(group_fairness(df), name='lottery_with_gaming')

### First Come First Served
With gaming, those with highest desire "come first", and so those with the most desire get the sheos

In [10]:
env.restock(num_shoes = 1000)        # reset environment to 100 shoes with no one purchased yet
env.run_first_come_with_gaming()

df = env.get_fake_df_firstcome()
#get_utilities(df)
#group_fairness(df)
first_with_gaming = pd.Series(group_fairness(df), name='first_with_gaming')

### Invitation Only
All influencers are invited, and the rest of the spots are randomly selected from those with loyalty status

In [11]:
env.restock(num_shoes = 1000)
env.run_invitation_with_gaming()
df = env.get_fake_df_loyal()
#get_utilities(df)
#group_fairness(df)
invite_with_gaming = pd.Series(group_fairness(df), name='invite_with_gaming')

## Change cap to 1
Change the cap to one to prevent resellers get too many shoes and ensure the fairness

In [12]:
env.restock(1000)
env.run_lottery_with_gaming(cap=1)  # run lottery and get dataframe of all results
df = env.get_fake_df()
#get_utilities(df)
#group_fairness(df)
lottery_cap1_with_gaming = pd.Series(group_fairness(df), name='lottery_cap1_with_gaming')

## Authentification

With auth, it's harder for reseller get fake acccounts, so they only have 1 more

In [13]:
env.restock(1000)
env.run_lottery_with_gaming_auth(cap=2)  # run lottery and get dataframe of all results
df = env.get_fake_df_auth()
#get_utilities(df)
#group_fairness(df)
lottery_authen_with_gaming = pd.Series(group_fairness(df), name='lottery_authen_with_gaming')

In [14]:
env.restock(1000)
env.run_lottery_with_gaming_auth(cap=1)  # run lottery and get dataframe of all results
df = env.get_fake_df_auth()
#get_utilities(df)
#group_fairness(df)
lottery_authen_cap1_with_gaming = pd.Series(group_fairness(df), name='lottery_authen_cap1_with_gaming')

## Combine three distribution methods

Since it's easier for resellers to use bot online, shoes will be distributed in a proportion that fewer percentage of them can be bought online to prevent that

In [15]:
env.restock(200)
env.run_lottery_with_gaming_auth()
env.restock_without_reset(500)
env.run_first_come_with_gaming()
env.restock_without_reset(300)
env.run_invitation_with_gaming()
df = env.get_real_fake_df()
#get_utilities(df)
#group_fairness(df)
combined_distribution_with_gaming = pd.Series(group_fairness(df), name='combined_distribution_with_gaming')

In [20]:
final_result = pd.DataFrame([lottery_no_gaming, first_no_gaming, invite_no_gaming,lottery_with_gaming,first_with_gaming,invite_with_gaming,lottery_cap1_with_gaming,lottery_authen_with_gaming,lottery_authen_cap1_with_gaming,combined_distribution_with_gaming])

In [35]:
shoe_person_col = [col for col in final_result.columns if 'num_got_shoes' in col]
shoe_number_col = [col for col in final_result.columns if 'shoes_bought' in col]
#final_result_shoes_number = final_result[['']]

In [36]:
shoe_number_col

['average_shoes_bought',
 'abnormal_shoes_bought',
 'special_shoes_bought',
 'wealthy_shoes_bought',
 'influencer_shoes_bought',
 'reseller_shoes_bought']

In [38]:
final_result_shoe_bought = final_result[shoe_number_col + shoe_person_col]

In [41]:
final_result_shoe_bought.reset_index(inplace=True)

In [48]:
pd.melt(final_result_shoe_bought, id_vars=['index'],value_vars = shoe_number_col)

,index,variable,value
0,lottery_no_gaming,average_shoes_bought,587.0
1,first_no_gaming,average_shoes_bought,776.0
2,invite_no_gaming,average_shoes_bought,255.0
3,lottery_with_gaming,average_shoes_bought,97.0
4,first_with_gaming,average_shoes_bought,277.0
5,invite_with_gaming,average_shoes_bought,123.0
6,lottery_cap1_with_gaming,average_shoes_bought,163.0
7,lottery_authen_with_gaming,average_shoes_bought,257.0
8,lottery_authen_cap1_with_gaming,average_shoes_bought,293.0
9,combined_distribution_with_gaming,average_shoes_bought,184.0


In [26]:
final_result.to_csv('project2_results.txt',sep = '\t')

In [32]:
final_result

,Consumer utility,Seller utility,average_num_got_shoes,average_percent_got_shoes,average_shoes_bought,average_shoes_per_person,abnormal_num_got_shoes,abnormal_percent_got_shoes,abnormal_shoes_bought,abnormal_shoes_per_person,...,wealthy_shoes_bought,wealthy_shoes_per_person,influencer_num_got_shoes,influencer_percent_got_shoes,influencer_shoes_bought,influencer_shoes_per_person,reseller_num_got_shoes,reseller_percent_got_shoes,reseller_shoes_bought,reseller_shoes_per_person
lottery_no_gaming,0.418593,0.158268,391.0,0.043444,587.0,0.065222,46.0,0.046,68.0,0.068,...,62.0,0.062,23.0,0.046,23.0,0.046,110.0,0.0440,220.0,0.0880
first_no_gaming,0.897413,0.083707,511.0,0.056778,776.0,0.086222,57.0,0.057,80.0,0.080,...,91.0,0.091,25.0,0.050,25.0,0.050,0.0,0.0000,0.0,0.0000
invite_no_gaming,0.458982,0.693890,166.0,0.018444,255.0,0.028333,16.0,0.016,24.0,0.024,...,49.0,0.049,500.0,1.000,500.0,1.000,79.0,0.0316,158.0,0.0632
lottery_with_gaming,0.094190,0.420802,66.0,0.007333,97.0,0.010778,11.0,0.011,14.0,0.014,...,15.0,0.015,1.0,0.002,1.0,0.002,430.0,0.1720,859.0,0.3436
first_with_gaming,0.423336,0.326512,182.0,0.020222,277.0,0.030778,18.0,0.018,23.0,0.023,...,39.0,0.039,12.0,0.024,12.0,0.024,318.0,0.1272,635.0,0.2540
invite_with_gaming,0.391721,0.760000,83.0,0.009222,123.0,0.013667,14.0,0.014,22.0,0.022,...,9.0,0.009,500.0,1.000,502.0,1.004,172.0,0.0688,344.0,0.1376
lottery_cap1_with_gaming,0.102155,0.412060,163.0,0.018111,163.0,0.018111,19.0,0.019,19.0,0.019,...,19.0,0.019,7.0,0.014,7.0,0.014,787.0,0.3148,787.0,0.3148
lottery_authen_with_gaming,0.203738,0.333037,177.0,0.019667,257.0,0.028556,18.0,0.018,30.0,0.030,...,25.0,0.025,11.0,0.022,11.0,0.022,335.0,0.1340,669.0,0.2676
lottery_authen_cap1_with_gaming,0.193456,0.350806,293.0,0.032556,293.0,0.032556,28.0,0.028,28.0,0.028,...,24.0,0.024,25.0,0.050,25.0,0.050,622.0,0.2488,622.0,0.2488
combined_distribution_with_gaming,0.428623,0.620640,122.0,0.013556,184.0,0.020444,14.0,0.014,20.0,0.020,...,14.0,0.014,305.0,0.610,307.0,0.614,235.0,0.0940,469.0,0.1876
